In [1]:
%matplotlib inline
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [31]:
from consts import sensor_day_seconds as day_second
from consts import sensor_idd as idd
from consts import sensor_idds_uniq as idds_uniq
from consts import sensor_idds_uniq_sorted as idds_uniq_sorted
from consts import sensor_a_temperature as a_temperature
from consts import sensor_s_temperature as s_temperature
from consts import sensor_unix_column as unix_column
from consts import sensor_idd as idd
epoch_length = 30

In [4]:
input_file = open('dict_list1.pkl','rb')
input_list = pickle.load(input_file)
input_file.close()

In [5]:
# 将epoch epoch_len epoch_time 都flat化
len_list = []
time_stamp_list = []
epoch_list = []
for start_time in input_list:
    # print(len(start_time[0]))
    for dict in start_time[0]:
        len_list.append(len(dict))
        epoch_list.append(dict)
    start_point = start_time[1]
    time_stamps = np.arange(start_point,start_point+day_second+epoch_length,epoch_length)
    time_stamp_list.extend(time_stamps[:-1])
len_array = np.array(len_list)
time_stamp_array = np.array(time_stamp_list)
epoch_array = np.array(epoch_list)

In [ ]:
index1 = np.where(len_array > 40)[0]
maybe_array = epoch_array[index1]

In [ ]:
# init maybe_dict 
maybe_dict = {}
for i in idds_uniq:
    maybe_dict[i] = 0
print(maybe_dict)


In [ ]:
for item in maybe_array:
    uniq_item = np.unique(item)
    for j in uniq_item:
        maybe_dict[j] += 1
print(maybe_dict)

In [ ]:
maybe_dict_sorted = sorted(maybe_dict.items(),key=lambda kv:(kv[1],kv[0]),reverse=True)
maybe_dict_sorted = np.array(maybe_dict_sorted)

In [ ]:
sub_sensor_list = maybe_dict_sorted.T[0][:28]
print(sub_sensor_list)

In [ ]:
maybe_dict_sorted.T[1][:40]

In [ ]:
# 测试sensor数量为多少合适
def get_sub_index(sensor_count):
    sub_sensor_list = maybe_dict_sorted.T[0][:sensor_count]
    sub_index = []
    for i, item in enumerate(epoch_array):
        uniq_item = np.unique(item)
        diff = np.setdiff1d(sub_sensor_list,uniq_item)
        if len(diff) == 0:
            sub_index.append(i)
    return len(sub_index),sub_index

In [ ]:
# sensor数量与合适epoch之间的关系
sensor_count_list = np.linspace(10,50,41,dtype=int)
sub_index_list = []
for i in sensor_count_list:
    tmp = get_sub_index(i)
    sub_index_list.append(tmp)
    print(i,tmp[0])

In [ ]:
sub_index_array = np.array(sub_index_list)

In [6]:
def get_length_count(window):
    for item in sub_index_array:
        zeros = np.zeros_like(epoch_array)
        zeros[item[1]] = 1
        sum = 0
        for j in range(len(zeros)):
            tmp = zeros[j:j+window]
            if 0 not in tmp:
                sum += 1
        print(sum)

In [ ]:
output_file = open('sub_index_array.pkl','wb')
pickle.dump(sub_index_array,output_file)
output_file.close()

In [7]:
input_file = open('sub_index_array.pkl','rb')
sub_index_array = pickle.load(input_file)
input_file.close()

In [8]:
get_length_count(7)

148328
132212
112581
98628
83055
70682
60151
53141
46963
38975
32943
27983
24729
21201
18281
16287
13963
12151
10287
8737
7624
6181
5411
4335
3961
3655
3141
2731
2369
1451
1222
1020
948
369
344
147
134
119
106
0
0


In [28]:
def drop_useless_sensor(df,sensor_list):
    # print(sensor_list)
    for i in df[idd].values:
        if i not in sensor_list:
            df = df[df[idd] != i]
    # print(len(df))
    if len(df) < len(sensor_list):
        return None
    return df
t = drop_useless_sensor(ddf_list[indices[3]],idds_uniq_sorted[:19])
print(len(t))

19


In [30]:
t = t.sort_values(by=idd)

In [37]:
t[s_temperature].values

array([10.412, 10.412, 12.35 , 11.6  , 12.725,  8.85 , 10.725, 11.037,
       11.287, 11.037, 10.85 , 10.1  , 10.725, 11.037, 10.6  , 10.35 ,
       10.662, 11.1  ,  9.475])

In [36]:
def get_transform_matrix(sensor_list):
    

,Station ID,Year,Month,Day,Hour,Minute,Second,Time since the epoch [s],Sequence Number,Ambient Temperature,Surface Temperature,Relative Humidity [%]
90482,8,2006,12,4,20,53,43,1165262023,21394,10.93,10.412,88.940
46039,11,2006,12,4,20,53,43,1165262023,51370,11.24,10.412,87.241
85747,17,2006,12,4,20,53,57,1165262037,12655,11.39,12.350,86.144
83293,34,2006,12,4,20,53,39,1165262019,1160,11.13,11.600,85.988
89434,35,2006,12,4,20,53,35,1165262015,1073,11.47,12.725,88.168
87287,46,2006,12,4,20,53,43,1165262023,9555,11.02,8.850,91.101
28333,49,2006,12,4,20,53,39,1165262019,30307,11.26,10.725,89.338
30960,51,2006,12,4,20,53,53,1165262033,33052,11.34,11.037,87.725
44224,55,2006,12,4,20,53,42,1165262022,833,NaN,11.287,NaN
62313,69,2006,12,4,20,53,36,1165262016,30070,NaN,11.037,NaN


In [53]:
for i,indice in enumerate(t[idd].values):
    flat_matrix[indice_dict[indice]] = t[s_temperature].values[i]

In [57]:
flat_matrix.reshape(10,10)

array([[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   , 12.35 ,  0.   ,  0.   ,
         0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  8.85 ,
        10.412,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , 11.6  ,
         0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        10.725,  0.   ],
       [ 0.   , 11.037, 10.412, 10.85 , 10.725,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ],
       [ 0.   ,  0.   , 11.287,  0.   , 10.1  ,  0.   , 12.725,  0.   ,
        11.037,  0.   ],
       [11.1  ,  0.   , 10.35 ,  0.   ,  0.   , 10.662,  0.   ,  0.   ,
         0.   ,  0.   ],
       [ 0.   , 11.037,  0.   ,  0.   ,  9.475,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ],
       [ 0.   , 10.6  ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ]])

In [52]:
flat_matrix = np.zeros(100)

In [43]:
indice_dict = {63:10,23:14,17:15,47:20,7:21,27:22,26:24,14:25,121:26,46:27,8:28,111:32,75:33,57:34,59:35,40:36,34:37,61:41,71:42,65:43,82:45,62:46,76:48,51:51,11:52,70:53,49:54,60:57,84:60,81:61,55:62,72:63,73:64,35:66,31:67,69:68,79:69,103:70,80:71,96:72,97:75,100:76,33:77,19:78,93:79,109:80,87:81,104:84,89:91,88:92,98:94}

In [47]:
for i in indice_dict:
    flat_matrix[indice_dict[i]] = i

In [49]:
flat_matrix.reshape(10,10)

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 63.,   0.,   0.,   0.,  23.,  17.,   0.,   0.,   0.,   0.],
       [ 47.,   7.,  27.,   0.,  26.,  14., 121.,  46.,   8.,   0.],
       [  0.,   0., 111.,  75.,  57.,  59.,  40.,  34.,   0.,   0.],
       [  0.,  61.,  71.,  65.,   0.,  82.,  62.,   0.,  76.,   0.],
       [  0.,  51.,  11.,  70.,  49.,   0.,   0.,  60.,   0.,   0.],
       [ 84.,  81.,  55.,  72.,  73.,   0.,  35.,  31.,  69.,  79.],
       [103.,  80.,  96.,   0.,   0.,  97., 100.,  33.,  19.,  93.],
       [109.,  87.,   0.,   0., 104.,   0.,   0.,   0.,   0.,   0.],
       [  0.,  89.,  88.,   0.,  98.,   0.,   0.,   0.,   0.,   0.]])

In [16]:
def handle_df(ddf_list,indice,sensor_list):
    df = ddf_list[indice]
    df = df.drop_duplicates(idd)
    df = drop_useless_sensor(df,sensor_list)
    print(len(df))

handle_df(ddf_list,indices[0],idds_uniq_sorted[:19])

19


In [21]:
def convert_df_to_dm(df,indice,sensor_list):
    # print(sensor_list)
    print(len(sensor_list))
    df = ddf_list[indice]
    print(len(df))
    df = df.drop_duplicates(idd)
    print(len(df))
    df = drop_useless_sensor(df,sensor_list)
    print(len(df))
    

convert_df_to_dm(ddf_list,indices[0],idds_uniq_sorted[:19])

19
37
36
19


In [22]:
def get_data_matrixs(sensor_count,window,ddf_list = None,sub_index_array = None):
    print(idds_uniq_sorted[:sensor_count])
    tmp = sub_index_array[sensor_count-10][1]
    zeros = np.zeros_like(epoch_array)
    zeros[tmp] = 1

    sum_list = []
    for i in range(len(zeros)):
        tmp1 = zeros[i:i+window]
        if 0 not in tmp1:
            sum_list.append(i)
            
    return sum_list
indices = get_data_matrixs(19,7,ddf_list,sub_index_array)

[103, 51, 87, 69, 70, 35, 34, 89, 96, 104, 11, 46, 17, 76, 97, 55, 73, 8, 49]


In [104]:
df = ddf_list[indices[0]]

In [40]:
idds_uniq_sorted[0]

103

In [52]:
df = df.drop_duplicates(idd)

In [8]:
for i in sub_index_array:
    print(i[0])

346050
335675
318371
306654
284198
268521
257301
246621
235724
226255
215845
206646
198888
189089
177260
171293
161414
154213
143979
132642
123279
114319
104536
93387
83788
80676
71524
67828
62249
45123
41373
37263
33852
16125
14077
8054
7153
7021
4894
0
0


In [ ]:
maybe_dict_sorted.T[0][:40]

In [ ]:
df = pd.read_hdf('day1.h5',str(start_list[1]))

In [10]:
start_list = np.arange(1162393768,1178747998+day_second,day_second)

In [11]:
ddf_list = []
for start_point in start_list:
    time_stamps = np.arange(start_point,start_point+day_second+epoch_length,epoch_length)
    sub_df_list = []
    df = pd.read_hdf('day1.h5',str(start_point))
    for i in range(len(time_stamps)-1):
        sub_df = df[(df[unix_column] > time_stamps[i]) & (df[unix_column] <= time_stamps[i+1])]
        sub_df_list.append(sub_df)
    ddf_list.extend(sub_df_list)

In [12]:
ddf_list[sub_index_array[0][1][1]]

,Station ID,Year,Month,Day,Hour,Minute,Second,Time since the epoch [s],Sequence Number,Ambient Temperature,Surface Temperature,Relative Humidity [%]
70645,8,2006,11,27,14,19,15,1164633555,14611,12.74,11.787,66.492
26185,11,2006,11,27,14,19,19,1164633559,29918,12.55,14.850,72.454
70209,14,2006,11,27,14,19,21,1164633561,11760,12.02,9.725,73.431
66903,17,2006,11,27,14,19,13,1164633553,21043,12.84,13.912,70.046
45042,19,2006,11,27,14,19,0,1164633540,21015,11.98,10.912,73.449
77199,23,2006,11,27,14,19,12,1164633552,23921,12.10,10.725,74.674
78066,26,2006,11,27,14,19,5,1164633545,46311,12.63,11.662,72.311
46764,31,2006,11,27,14,19,13,1164633553,13393,13.03,13.850,72.149
49688,33,2006,11,27,14,19,21,1164633561,25014,12.09,11.537,74.516
63331,34,2006,11,27,14,19,24,1164633564,11564,12.10,11.350,71.142


In [10]:
print(len(ddf_list)*len(ddf_list[0]))

550080


In [11]:
tmp_array.max()

NameError: name 'tmp_array' is not defined

In [ ]:
time_stamps

In [ ]:
print(epoch_length)

In [ ]:
sub_index_array[0][1][4]